<a href="https://colab.research.google.com/github/clarkde5/jams-pub/blob/main/colabs/jams-env-doctr-copies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Environment**

## **Bootstrap public repo**

In [ ]:
from pathlib import Path

if not Path(f"/content").is_dir():
  %cd ~
else:
  %cd /content

if not Path(f"src/jams-pub").is_dir():
  !git clone https://github.com/clarkde5/jams-pub src/jams-pub
else:
  print("src/jams-pub already exists")

## **Pull private resources**

In [ ]:
runOptional = True
def CallPrivateEnvSetup():
  from pathlib import Path

  home = str(Path.home())

  try:
    from google.colab import userdata
    try:
      rsa_private_key = userdata.get('PrivateKey')
    except:
      rsa_private_key = ""
      print("PrivateKey is required, but not found in secrets / colab userdata")
      return
  except:
    print("google.colab could not be loaded")
    if not Path(f"{home}/.ssh/id_rsa").is_file():
      print("rsa key does not exist, please create manually")
      return
    else:
      print("rsa key already exists")

  if not Path(f"{home}/.ssh/id_rsa").is_file():
    !python src/jams-pub/env-setup.py "{rsa_private_key}"

  if not Path(f"{home}/.ssh/id_rsa").is_file():
    print("env-setup failed to create rsa key")
    return

  if not Path(f"src/jams").is_dir():
    !git clone git@github.com:clarkde5/jams.git src/jams
  else:
    print("src/jams already exists")

if __name__ == "__main__":
  if runOptional == True:
    CallPrivateEnvSetup()
  else:
    print("Not running optional cell")

# **Parse Lease Output**

**NOTE:** Requires [Pull private resources](https://colab.research.google.com/github/clarkde5/jams-pub/blob/main/colabs/jams-env-doctr.ipynb#scrollTo=CyQkr3hoftql)

In [ ]:
runOptional = True
def getContractsForPage(page_idx,page):
  import re
  StartingPlaceFound = page_idx != 0
  contracts = []
#   if page_idx == 0:
#     print(page)


  for block in page["blocks"]:
    # if page_idx == 0:
        # print('-----1--block Start-----' )
        # print(block)
        # print('-----1--block end-----' )
    for line in block["lines"]:
      for word_idx,word in enumerate(sorted(line["words"], key = lambda x: x["geometry"][0][1])):
        # print("---1--word value---" + word["value"])
        if re.search("Unit",word["value"]):
            StartingPlaceFound = True

        # print("---2--word value---" + word["value"])
        if not StartingPlaceFound:
            continue

        # print("---3--word value---" + word["value"])
        if re.search("Contract",word["value"]):
            # print("---4--word value---" + word["value"])
            if len(line["words"])>=3:
                contract_number_word = line["words"][2]
                # print("---5--contract_number_word ---" +  contract_number_word["value"])
                if re.search("\d{8}",contract_number_word["value"]):
                    contracts.append({"contract_number": contract_number_word["value"], "pdf_y": contract_number_word["geometry"][0][1]+page_idx, "page": page_idx+1})
        else:
            continue


  return contracts

def getSerialNumbersForPage(page_idx,page):
  import re
  StartingPlaceFound = page_idx != 0
  serialNumbers = []

  for block in page["blocks"]:
    for line_idx,line in enumerate(block["lines"]):
      sorted_words = sorted(line["words"], key = lambda x: x["geometry"][0][1])
      for word_idx,word in enumerate(sorted_words):
        if re.search("Unit",word["value"]):
          StartingPlaceFound = True

        if not StartingPlaceFound:
          continue

        if re.search("Contract",word["value"]):
            if len(line["words"])>=3:
                contract_number_word = line["words"][2]
                if re.search("\d{8}",contract_number_word["value"]):
                    serial_number_word = block["lines"][line_idx+1]["words"][0]
                    # print(serial_number_word["value"])
                    if serial_number_word["value"][4] == 'O':
                        serial_number_word["value"] = serial_number_word["value"][:4] + "0" + serial_number_word["value"][5:]
                    serialNumbers.append({"serial_number": serial_number_word["value"], "pdf_y": serial_number_word["geometry"][0][1]+page_idx, "page": page_idx+1})
        else:
          continue

  return serialNumbers

# def getPaymentDue(page_idx,page):
#   import re
#   StartingPlaceFound = page_idx != 0
#   PleaseFound = False
#   prices = []

#   for block in page["blocks"]:
#     for line in block["lines"]:
#       for word_idx,word in enumerate(sorted(line["words"], key = lambda x: x["geometry"][0][1])):
#         if re.search("PLEASE",word["value"]):
#           PleaseFound = True

#         if not PleaseFound:
#           continue

#         if re.search("CURRENT",word["value"]):
#           StartingPlaceFound = True

#         if not StartingPlaceFound:
#           continue

#         if re.search("-{0,1}\d+\.\d{2}",word["value"]):
#           prices.append({"price": word["value"], "pdf_y": word["geometry"][0][1]+page_idx, "page": page_idx+1})
#         else:
#           continue

#   return prices

def main():
  import json
  import re

  f = open('C:\Vaidhy\JAMS\Solution\Aug23copies-docTR.json')
  data = json.load(f)

  contracts = []
  serialNumbers = []
  paymentDue = []

  for page_idx,page in enumerate(data["pages"]):
    contracts += getContractsForPage(page_idx,page)
    serialNumbers += getSerialNumbersForPage(page_idx,page)
    # paymentDue += getPaymentDue(page_idx,page)
#   json_formatted_str = json.dumps(contracts, indent=2)
#   print(json_formatted_str)
  json_formatted_str = json.dumps(serialNumbers, indent=2)
  print(json_formatted_str)
  # contractSerials = contracts + serialNumbers + paymentDue
  # contractSerials = sorted(contractSerials, key = lambda x: x["pdf_y"])
  # currentPage = 0

  # for contractSerialPair in contractSerials:
  #   if contractSerialPair["page"] != currentPage:
  #     currentPage = contractSerialPair["page"]
  #     print("\r\nPage: " + str(currentPage) + "\r\n")

  #   if "contract_number" in contractSerialPair:
  #     print("Contract Number: " + contractSerialPair["contract_number"])
  #   elif "serial_number" in contractSerialPair:
  #     print("Serial Number: " + contractSerialPair["serial_number"])
  #   elif "price" in contractSerialPair:
  #       print("Due: " + contractSerialPair["price"])
  #   else:
  #     print("Error unknown pair: " + contractSerialPair)

if __name__ == "__main__":
  if runOptional == True:
    main()
  else:
    print("Not running optional cell")